In [83]:
import numpy as np # linear algebra
import pandas as pd
import re
import string
import gensim
import enchant
import enchant.checker
from enchant.checker.CmdLineChecker import CmdLineChecker
from collections import Counter
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
#train = train.sample(frac=1)

In [3]:
labels = train.columns[2:]
tags = []
for i in range(len(train)):
    t = []
    for lab in labels:
        if train[lab][i]==1:
            t.append(lab)
    if not t:
        t.append('clean')
    tags.append(','.join(t))
train['tags']=tags
train.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,tags
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,clean
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,clean


In [44]:

glove2word2vec(glove_input_file="glove.6B.200d.txt", word2vec_output_file="gensim_glove_vectors.txt")

model = gensim.models.KeyedVectors.load_word2vec_format('gensim_glove_vectors.txt',binary=False)

words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))





In [75]:
def spell_checker(text):
    
    chkr = enchant.checker.SpellChecker("en_US")
    chkr.set_text(text)
    for err in chkr:
        #print (err.word)
        #sug = err.suggest()[0]
        #sug = err.suggest()[0]
        err.replace(correction(err.word))
        #err.replace(sug)
        #print (sug)
        #print (correction(err.word))

    c = chkr.get_text()#returns corrected text
    return c

In [151]:
wnl = WordNetLemmatizer()
stop = set(stopwords.words('english'))

def clean_text(text):
    
    text = " " + text + " "
    
    text= re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"," ",str(text.lower())) #IP's
    text= re.sub("http://.*com"," ",str(text)) #Links
    text =re.sub("\\n","",text)
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub("\'s", " ", text)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub(","," ",text)
    
    text= re.sub(r'\d{2}:\d{2}', ' ', text) #Removing timestamp
    text = text.replace('(utc)','')
    
    #Punctuations
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
    
    #Removing stopwords
    text = " ".join([i for i in text.split() if i not in stop])
    
    #Lemmatisation
    text = " ".join([wnl.lemmatize(x,pos='v') for x in text.split()])
    #text = " ".join([wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','v'] else wnl.lemmatize(i) for i,j in pos_tag(word_tokenize(text))])
    
    #Spelling Correction
    #text = spell_checker(text)
    
    
   
    return text

In [152]:
train['filtered_text']=train['comment_text'].apply(lambda x:clean_text(x))

In [154]:
tox = train[train['tags']!='clean'].reset_index(drop=True)
rand = tox.sample(50).reset_index(drop=True)
for i in range(len(rand)):
    print (i,tox['tags'][i],'\n')
    print (tox.comment_text[i])
    print ('\n')
    print (tox.filtered_text[i])
    print ('\n')

0 toxic,severe_toxic,obscene,insult 

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK


cocksucker piss around work


1 toxic 

Hey... what is it..
@ | talk .
What is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?

Ask Sityush to clean up his behavior than issue me nonsensical warnings...


hey talk exclusive group wp talibans good destroy self appoint purist gang one ask question abt anti social destructive non contribution wp ask sityush clean behavior issue nonsensical warn


2 toxic 

Bye! 

Don't look, come or think of comming back! Tosser.


bye look come think comming back tosser


3 toxic,obscene,insult,identity_hate 

You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/

In [155]:
train.to_csv('filtered_train.csv',index=False)